In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('ucla_results_SDML_exp_spectral', sep = '\t', index_col=0)

In [3]:
data

,normalization1,normalization2,features,selection,machine_learning,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,wbysqdist,spectral,degrees,vartreshold,SDML,0.698936,0.250953,"{'metric__balance': 0.1, 'metric__use_cov': Fa...",0.697,0.157293,[ 0.5 0.88 0.44 0.9 0.75 0.8 0.6 0....
1,wbysqdist,spectral,degrees,origS,SDML,0.698936,0.250953,"{'metric__balance': 0.1, 'metric__use_cov': Fa...",0.697,0.157293,[ 0.5 0.88 0.44 0.9 0.75 0.8 0.6 0....


In [20]:
data['eval_roc_auc_scores'][0], data.eval_roc_auc_mean

('[ 0.5   0.88  0.44  0.9   0.75  0.8   0.6   0.55  0.7   0.85]', 0    0.697
 1    0.697
 Name: eval_roc_auc_mean, dtype: float64)

In [4]:
d = pd.read_csv('ucla_results_LMNN_inn_spectral', sep ='\t', index_col = 0)

In [19]:
d['eval_roc_auc_scores'][0] ,dd.eval_roc_auc_mean

('[ 0.46666667  0.96        0.64        0.9         0.65        0.8         0.65\n  0.8         0.75        0.9       ]',
 0    0.736
 1    0.736
 Name: eval_roc_auc_mean, dtype: float64)

In [11]:
dd = pd.read_csv('ucla_results_SDML_inn_spectral', sep = '\t', index_col = 0)

In [17]:
dd['eval_roc_auc_scores'][0], dd.eval_roc_auc_mean

('[ 0.5   0.8   0.56  0.9   0.65  0.85  0.6   0.8   0.8   0.9 ]', 0    0.736
 1    0.736
 Name: eval_roc_auc_mean, dtype: float64)

In [21]:
dd

,normalization1,normalization2,features,selection,machine_learning,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,wbysqdist,spectral,degrees,vartreshold,SDML,0.731383,0.230077,"{'metric__use_cov': False, 'metric__balance': ...",0.736,0.138434,[ 0.5 0.8 0.56 0.9 0.65 0.85 0.6 0....
1,wbysqdist,spectral,degrees,origS,SDML,0.731383,0.230077,"{'metric__use_cov': False, 'metric__balance': ...",0.736,0.138434,[ 0.5 0.8 0.56 0.9 0.65 0.85 0.6 0....


In [23]:
import numpy as np

In [34]:
a = np.random.randint(0,2,25).reshape(5,5)

In [35]:
a

array([[1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 1, 0, 1]])

In [36]:
b =  np.random.randint(1,10,25).reshape(5,5)

In [37]:
a/ b.dot(b.T)

array([[ 0.00934579,  0.00970874,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.0052356 ,  0.01315789],
       [ 0.        ,  0.        ,  0.00534759,  0.00518135,  0.        ],
       [ 0.00740741,  0.        ,  0.        ,  0.00359712,  0.00645161],
       [ 0.        ,  0.        ,  0.00806452,  0.        ,  0.00704225]])

In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold

from norms import OrigN, OrigS, SpectralNorm, WbysqDist, BinarNorm
from sklearn.feature_selection import VarianceThreshold

import os
import pandas as pd
import numpy as np

from MetricTransform import MetricTransform

from reskit.core import DataTransformer, MatrixTransformer, Pipeliner

from reskit.features import degrees, bag_of_edges, closeness_centrality, betweenness_centrality


from convert import convert
from load_data import load_ucla

from norms import OrigN, OrigS, SpectralNorm, WbysqDist, BinarNorm

path_ucla = '../../../Connectomics/Autism/Data/'

matrix_ucla, target_ucla, xyz = load_ucla(path_ucla)
X = {}
X['data'] = matrix_ucla
X['dist'] = xyz
y = target_ucla




param_LMNN = {'model': 'LMNN',
              'mode': 'inner_product',
                'max_iter': 100,
              'min_iter':10,
              'learn_rate':  1e-7,
              'regularization': 0.5,
              'conv_tol': 1e-5,
              'k': 5,
             }
normalizers1 = [('wbysqdist', WbysqDist())]
normalizers2 = [('spectral', SpectralNorm())]

featurizers = [('degrees', MatrixTransformer(degrees))]

selection = [('origS', OrigS())]


pipe_lmnn = Pipeline(steps=[('metric', MetricTransform(**param_LMNN)), ('clf', SVC(kernel='precomputed'))])

metric_trans_learn = [('LMNN', pipe_lmnn)]


steps = [('normalization1', normalizers1),
         ('normalization2', normalizers2),
         ('features', featurizers),
         ('selection', selection),
         ('machine_learning', metric_trans_learn)]



param_grid = {'LMNN': {'metric__max_iter':[100, 500],
                       'metric__min_iter':[10],
                  'metric__learn_rate': np.linspace(1e-10, 1e-2, 3),
                  'metric__regularization': [0.1, 0.25, 0.65, 0.9],
                  'metric__conv_tol': [0.001, 1e-5],
                  'metric__k': [ 25,  55],
                  'clf__C':[1e-7, 1e-2, 0.1, 1, 10 ]},
    }
    
    


grid_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
eval_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

pipeliner = Pipeliner(steps=steps, grid_cv=grid_cv, 
                      eval_cv=eval_cv, param_grid=param_grid)




UCLA Autism data shape                   : (94, 264, 264) 
UCLA Autism target variable shape        : (94,)


In [ ]:
table = pipeliner.get_results(X, y, scoring=['roc_auc'], caching_steps = ['normalization1','normalization2','features','selection'], logs_file = 'log/ucla_results_LMNN_corr.log')


Line: 1/1


In [ ]:
table.to_csv('ucla_results_LMNN_corr', sep = '\t')

In [ ]:
table

In [ ]:
print()